In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install transformers
!pip install datasets
!pip install nervaluate
!pip install accelerate -U
!pip install tf-keras


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
# Disable parallelism for tokenizers
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from tqdm import tqdm
import numpy as np
from nervaluate import Evaluator

2024-06-29 17:54:43.078989: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
from datasets import load_dataset, ClassLabel, Sequence
custom_headers = ["Id", "tags", "text"] 
dataset = load_dataset('csv', data_files={'train': ['../data/processed/small_medical_ner_first_10k.tsv']}, delimiter="\t",column_names=custom_headers)
dataset


DatasetDict({
    train: Dataset({
        features: ['Id', 'tags', 'text'],
        num_rows: 10001
    })
})

In [6]:
# Define label set and mapping
label_set = ['B-age', 'I-age', 'B-allergy_name', 'I-allergy_name', 'B-bmi', 'I-bmi', 'B-cancer', 'I-cancer', 'B-chronic_disease', 'I-chronic_disease', 'B-clinical_variable', 'I-clinical_variable', 'B-contraception_consent', 'I-contraception_consent', 'B-ethnicity', 'I-ethnicity', 'B-gender', 'I-gender', 'B-language_fluency', 'I-language_fluency', 'B-lower_bound', 'I-lower_bound', 'B-pregnancy', 'I-pregnancy', 'B-technology_access', 'I-technology_access', 'B-treatment', 'I-treatment', 'B-upper_bound', 'I-upper_bound', 'O']
label2id = {label: i for i, label in enumerate(label_set)}
id2label = {i: label for i, label in enumerate(label_set)}
label2id
id2label

{0: 'B-age',
 1: 'I-age',
 2: 'B-allergy_name',
 3: 'I-allergy_name',
 4: 'B-bmi',
 5: 'I-bmi',
 6: 'B-cancer',
 7: 'I-cancer',
 8: 'B-chronic_disease',
 9: 'I-chronic_disease',
 10: 'B-clinical_variable',
 11: 'I-clinical_variable',
 12: 'B-contraception_consent',
 13: 'I-contraception_consent',
 14: 'B-ethnicity',
 15: 'I-ethnicity',
 16: 'B-gender',
 17: 'I-gender',
 18: 'B-language_fluency',
 19: 'I-language_fluency',
 20: 'B-lower_bound',
 21: 'I-lower_bound',
 22: 'B-pregnancy',
 23: 'I-pregnancy',
 24: 'B-technology_access',
 25: 'I-technology_access',
 26: 'B-treatment',
 27: 'I-treatment',
 28: 'B-upper_bound',
 29: 'I-upper_bound',
 30: 'O'}

In [7]:
dataset['train'].num_rows


10001

In [8]:
tokenizer = AutoTokenizer.from_pretrained("medicalai/ClinicalBERT")
model = AutoModelForTokenClassification.from_pretrained("medicalai/ClinicalBERT", num_labels=len(label_set))


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Id', 'tags', 'text'],
        num_rows: 10001
    })
})

In [10]:
# Freeze all layers except the last one
for name, param in model.named_parameters():
    if 'classifier' not in name:  # Freeze all layers except classifier
        param.requires_grad = False

In [11]:

# Function to align labels with tokenized inputs
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [12]:
# Function to tokenize and align labels for token classification task
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [13]:
# Process dataset
all_tokens = []
all_ner_tags = []
for idx in tqdm(range(dataset['train'].num_rows)):
    tags = {}
    tokens = []
    ner_tags = []
    for position_and_tag in dataset['train'][idx]['tags'].split(','):
        start_position, end_position, tag = int(position_and_tag.split(':')[0]), int(position_and_tag.split(':')[1]), position_and_tag.split(':')[2]
        tags[(start_position-1, end_position-1)] = tag
    pre_tokenized_text = tokenizer(dataset['train'][idx]['text'], return_offsets_mapping=True)
    for idx, (start, end) in enumerate(pre_tokenized_text['offset_mapping']):
        if start != end:
            token = dataset['train'][idx]['text'][start:end]
            tokens.append(token)
            ner_tags.append(label2id.get(tags.get((start, end), 'O'), label2id['O']))
    all_tokens.append(tokens)
    all_ner_tags.append(ner_tags)

my_dict = {"tokens": all_tokens, "ner_tags": all_ner_tags}
processed_dataset = Dataset.from_dict(my_dict)
tokenized_datasets = processed_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=processed_dataset.column_names)

100%|██████████| 10001/10001 [00:11<00:00, 863.47it/s]


Map:   0%|          | 0/10001 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [14]:
# Load evaluation metric (e.g., seqeval)
import evaluate
metric = evaluate.load("seqeval")

# Function to compute metrics for token classification
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for (l, p) in zip(label, prediction) if l != -100] for label, prediction in zip(labels, predictions)]
    pred_labels = [[id2label[p] for (l, p) in zip(label, prediction) if l != -100] for label, prediction in zip(labels, predictions)]

    # Compute metrics
    results = metric.compute(predictions=pred_labels, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [15]:
import torch
torch.mps.set_per_process_memory_fraction(0.0)

In [16]:
# Define training arguments
args = TrainingArguments(
    "clinical-bert-last-layer-finetuned-ner",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

# Split dataset
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.20)

# Load model for token classification
model_checkpoint = "medicalai/ClinicalBERT"
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, id2label=id2label, label2id=label2id)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=DataCollatorForTokenClassification(tokenizer),
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

os.environ['PYTORCH_MPS_HIGH_WATERMARK_RATIO'] = '0.0'

# Clear cache
torch.cuda.empty_cache()
# Train the model
trainer.train()

/Users/shahzeb/.pyenv/versions/3.11.5/lib/python3.11/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at medicalai/ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/750 [00:00<?, ?it/s]

RuntimeError: Placeholder storage has not been allocated on MPS device!